In [1]:
import sys
pdefind_path = "/Users/pongpisit/Desktop/research/PDE-FIND/"
sys.path.insert(0, pdefind_path)
from misc import h5file

import jax.numpy as jnp
from scipy.optimize import curve_fit
from jaxfit import CurveFit
from levenberg_marquardt import lm as lm_curve_fit

import numpy as np
import pandas as pd
from scipy.io import loadmat
import sympy as sp
import matplotlib.pyplot as plt
import dedalus.public as d3

def log_like_value(prediction, ground):                                                                                                               
    nobs = float(ground.shape[0])
    nobs2 = nobs / 2.0
    ssr = np.sum(np.abs(ground - prediction)**2)
    llf = -nobs2 * np.log(2 * np.pi) - nobs2 * np.log(ssr / nobs) - nobs2
    return llf

def BIC_AIC(prediction, ground, nparams, reg_func = lambda x: x):
    nparams = reg_func(nparams)
    llf = log_like_value(prediction, ground)
    return -2*llf + np.log(ground.shape[0])*nparams, -2*llf + 2*nparams

In [2]:
# Parameters
Rx = 32*np.pi
Nx = 1024
Lx = Rx/Nx
dealias = 1.5
stop_sim_time = 100
timestepper = d3.SBDF4
acc = 16
timestep = 0.016/acc
dtype = np.float64

In [3]:
# Bases
xcoord = d3.Coordinate('x')
dist = d3.Distributor(xcoord, dtype=dtype)
xbasis = d3.RealFourier(xcoord, size=Nx, bounds=(Lx, Rx+Lx), dealias=dealias)

# Fields
u = dist.Field(name='u', bases=xbasis)

# Substitutions
dx = lambda A: d3.Differentiate(A, xcoord)

In [4]:
complexity = 4
X_pre, best_subsets, un, y_pre = h5file("./IPI_output_files/PMS_data.h5")
xaxis = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/Datasets/kuramoto_sivishinky.mat")['x']
if complexity == 3:
    # 0.9951430536784973 0.9966016871225765 0.9968154216196904
    a, b, c = np.abs(np.linalg.lstsq(X_pre[:, np.where(best_subsets[complexity-1]>0)[0]], y_pre, 
                                     rcond=None)[0].flatten())
    print(a, b, c)
elif complexity == 4:
    # 0.9952025550450707 0.0006388964618597726 0.9966536432642854 0.9968941888507773
    a, b, c, d = np.abs(np.linalg.lstsq(X_pre[:, np.where(best_subsets[complexity-1]>0)[0]], y_pre, 
                                        rcond=None)[0].flatten())
    print(a, b, c, d)

['X_pre', 'best_subsets', 'un', 'y_pre']
0.9952025550450707 0.0006388964618597726 0.9966536432642854 0.9968941888507773


In [5]:
# Problem
problem = d3.IVP([u], namespace=locals())
# a, b, c = (1.0, 1.0, 1.0)
if complexity == 3:
    problem.add_equation("dt(u) + a*dx(dx(u)) + b*dx(dx(dx(dx(u)))) = - c*u*dx(u)")
elif complexity == 4:
    problem.add_equation("dt(u) + a*dx(dx(u)) - b*dx(dx(dx(u))) + c*dx(dx(dx(dx(u)))) = - d*u*dx(u)")

In [6]:
hall_path = "./hall/1/hall_of_fame_2023-05-27_145328.068.csv"
equation = pd.read_csv(hall_path)['Equation'].iloc[7]; print(equation)
at1, at2 = sp.sympify(equation).atoms(sp.Number)
if at1 > at2: at1, at2 = at2, at1
pysr_params = np.array([at1, at1, at2], dtype=dtype)
def initial_function(x, a, b, c): return np.cos(a*x)*(c+np.sin(b*x))
def jax_initial_function(x, a, b, c): return jnp.cos(a*x)*(c+jnp.sin(b*x))
pysr_params

(cos(0.062976345 * x0) * (0.9813761 + sin(0.062976345 * x0)))


array([0.06297635, 0.06297635, 0.9813761 ])

In [7]:
lm_func = lambda t,p: np.cos(p[0,0]*t)*(p[2,0]+np.sin(p[1,0]*t))

recovered_params3 = lm_curve_fit(np.round(pysr_params.reshape(-1, 1), 1), 
                                 xaxis.flatten(), un[:, 0], 
                                 lm_func)[0].flatten()

# recovered_params1 and recovered_params2 gave similar results. Choose 1.
recovered_params2 = np.array(curve_fit(initial_function, xaxis.flatten(), un[:, 0], 
                                       p0=pysr_params, method='lm')[0])
recovered_params1 = np.array(CurveFit().curve_fit(jax_initial_function, xaxis.flatten(), un[:, 0], 
                                                  p0=pysr_params)[0])

pysr_params, recovered_params1, recovered_params2, recovered_params3

Using uniform weights for error analysis
**** Convergence in Parameters ****

LM fitting results:
----------------------------- 
parameter      = p1
fitted value   = 0.0594
standard error = 0.70 %
----------------------------- 
parameter      = p2
fitted value   = 0.0592
standard error = 0.70 %
----------------------------- 
parameter      = p3
fitted value   = 0.9879
standard error = 0.04 %
2023-12-23 13:48:33,315 jax._src.dispatch 0/1 WARNING :: Finished tracing + transforming jit(convert_element_type) in 0.0003159046173095703 sec
2023-12-23 13:48:33,322 jax._src.xla_bridge 0/1 INFO :: Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023-12-23 13:48:33,322 jax._src.xla_bridge 0/1 INFO :: Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
2023-12-23 13:48:33,323 jax._src.xla_bridge 0/1 INFO :: Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 

(array([0.06297635, 0.06297635, 0.9813761 ]),
 array([0.06263812, 0.06286727, 0.9839311 ]),
 array([0.06263812, 0.06286727, 0.9839311 ]),
 array([0.05936273, 0.05919549, 0.98790786]))

In [8]:
# pysr_params, recovered_params1, recovered_params2, recovered_params3
recovered_params = recovered_params1
# Initial conditions
x = dist.local_grid(xbasis)
# u['g'] = np.cos(x/16)*(1+np.sin(x/16))
# u['g'] = np.cos(recovered_params[0]*x)*(recovered_params[2]+np.sin(recovered_params[1]*x))
u['g'] = un[:, 0]

In [9]:
# load ref
data = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/Datasets/kuramoto_sivishinky.mat")
u_clean = data["uu"]
u_clean = u_clean.real
xaxis = data['x'].real.ravel()
taxis = data['tt'].real.ravel()

In [10]:
xaxis[1]-xaxis[0] == xaxis[0] == 32*np.pi/1024  and xaxis[-1] == 32*np.pi

True

In [11]:
# noise_lv = 30
# u_test = u_clean + 0.01*np.abs(noise_lv)*(u_clean.std())*np.random.randn(u_clean.shape[0], 
#                                                                          u_clean.shape[1])

In [12]:
# load cf
# cf_com3 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/ks_chaotic.mat")
# cf_com3 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_reproduce.mat")
cf_com3 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_simV2_adjustedIC_com3.mat")
cf_com3 = cf_com3['usol'][:, :]

# cf_com3 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_reproduce.mat")
# cf_com3 = cf_com3['usol'][1:, :]
# cf_com3 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_sim_adjustedIC_com3.mat")
# cf_com3 = cf_com3['usol'][1:, :]

# cf_com4 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_sim_adjustedIC_com4.mat")
# cf_com4 = cf_com4['usol'][1:, :]
cf_com4 = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_simV2_adjustedIC_com4.mat")
cf_com4 = cf_com4['usol'][:, :]

u_test = un.copy()
bic_com3 = BIC_AIC(cf_com3.reshape(-1, 1), u_test.reshape(-1, 1), 3)[0]
bic_com4 = BIC_AIC(cf_com4.reshape(-1, 1), u_test.reshape(-1, 1), 4)[0]
assert bic_com3 < bic_com4
bic_com3, bic_com4

(783460.7059873166, 811905.6349093843)

In [13]:
# 0.0018096646361430416
# 0.001809642483677698
np.linalg.norm(u_clean-cf_com3, 'fro'), np.linalg.norm(u_clean-cf_com3, np.inf)

(562.8375096054419, 327.49018941487003)

In [ ]:
noise_lv = 30
on_undenoised = False
if on_undenoised:
    np.random.seed(0)
    u_test = u_clean + 0.01*np.abs(noise_lv)*(u_clean.std())*np.random.randn(u_clean.shape[0], 
                                                                             u_clean.shape[1])
else:
    u_test = un.copy()
    
bic, aic = BIC_AIC(pred.reshape(-1, 1), u_test.reshape(-1, 1), complexity)
complexity, bic, aic

In [ ]:
noise_lv = 30
on_undenoised = True
if on_undenoised:
    np.random.seed(0)
    u_test = u_clean + 0.01*np.abs(noise_lv)*(u_clean.std())*np.random.randn(u_clean.shape[0], 
                                                                             u_clean.shape[1])
else:
    u_test = un.copy()
    
bic, aic = BIC_AIC(pred.reshape(-1, 1), u_test.reshape(-1, 1), complexity)
complexity, bic, aic

In [ ]:
##### Dedalus3 (pred = simulated_var.T) #####
### raw values un[:, 0] as IC *** ###
# (3, 939694.1722609766, 939662.8014867424) on denoised
# (4, 968190.4618845284, 968148.6341855495) on denoised
# (3, 950743.0252888124, 950711.6545145782) on undenoised
# (4, 978106.5687099883, 978064.7410110093) on undenoised
### pysr_params IC ###
# (3, 803185.643152019, 803154.2723777848) on denoised
# (4, 779213.6691787682, 779171.8414797892) on denoised
# (3, 821716.647611001, 821685.2768367669) on undenoised
# (4, 799477.7687019842, 799435.9410030053) on undenoised
### Adjusted IC ###
## params1 (Best adjusted IC) ***
# (3, 783458.9685251904, 783427.5977509562) on denoised
# (4, 811910.3272131429, 811868.4995141639) on denoised
# (3, 803400.5704714309, 803369.1996971967) on undenoised
# (4, 829826.8625591999, 829785.034860221) on undenoised
## params2 (Best adjusted IC)
# (3, 783458.9907923832, 783427.620018149) on denoised
# (4, 811910.3259884643, 811868.4982894853) on denoised
# (3, 803400.5911657464, 803369.2203915123) on undenoised
# (4, 829826.861417452, 829785.033718473) on undenoised
## params3
# (3, 886732.4482964904, 886701.0775222562) on denoised
# (4, 895679.9103753887, 895638.0826764097) on denoised
# (3, 900252.8444702567, 900221.4736960225) on undenoised
# (4, 908750.5194537467, 908708.6917547678) on undenoised

#### Unused

In [ ]:
# # load cf true
# cf = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/ks_chaotic.mat")
# cf_usol = cf['usol']
# ((cf_usol-u_clean)**2).mean(), BIC_AIC(cf_usol.reshape(-1, 1), u_clean.reshape(-1, 1), 3)

# # load cf
# cf = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_sim_com4.mat")
# cf = loadmat("/Users/pongpisit/Desktop/research/PDE-FIND/DeepHPMs/Matlab/KS_sim/ks_chaotic_sim_adjustedIC_com4.mat")
# cf_usol = cf['usol']
# ((cf_usol-u_clean)**2).mean(), BIC_AIC(cf_usol.reshape(-1, 1), u_clean.reshape(-1, 1), 3)

# print(((simulated_var.T-cf_usol)**2).mean())
# plt.figure(figsize=(6, 4))
# plt.pcolormesh(cf['x'].ravel(), cf['t'].ravel(), cf_usol.T, cmap='coolwarm', shading='gouraud', rasterized=True, clim=(-0.8, 0.8))
# plt.xlim(0, Rx)
# plt.ylim(0, stop_sim_time)
# plt.xlabel('x')
# plt.ylabel('t')
# plt.show()

In [ ]:
##### Chebfun (pred = cf_usol) #####
### pysr_params IC ###
# (3, 796688.596808223, 796657.2260339889) on denoised
# (4, 773154.477631851, 773112.649932872) on denoised
# --- #
# (3, 815671.8505524133, 815640.4797781791) on undenoised
# (4, 793886.9601944236, 793845.1324954446) on undenoised
### Adjusted IC ###
## params1
# (3, 829873.6793635229, 829842.3085892887) on denoised
# (4, 819650.1445381376, 819608.3168391587) on denoised
# (3, 846624.4711688751, 846593.1003946409) on undenoised
# (4, 837056.275441391, 837014.4477424121) on undenoised